In [7]:
from sklearn.datasets import load_iris, load_wine, load_breast_cancer, load_digits, fetch_openml
import pandas as pd
from IPython.display import clear_output

# Função para carregar os datasets
def carregar_dataset(nome_dataset):
    if nome_dataset == 'iris':
        data = load_iris()
        X, y = pd.DataFrame(data.data, columns=data.feature_names), data.target
        class_names = data.target_names
    
    elif nome_dataset == 'wine':
        data = load_wine()
        X, y = pd.DataFrame(data.data, columns=data.feature_names), data.target
        class_names = data.target_names
    
    elif nome_dataset == 'breast_cancer':
        data = load_breast_cancer()
        X, y = pd.DataFrame(data.data, columns=data.feature_names), data.target
        class_names = data.target_names
    
    elif nome_dataset == 'digits':
        data = load_digits()
        X, y = pd.DataFrame(data.data, columns=[f"pixel_{i}" for i in range(data.data.shape[1])]), data.target
        class_names = [str(i) for i in range(10)]
    
    elif nome_dataset == 'banknote_authentication':
        data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt", header=None)
        X = data.iloc[:, :-1]
        y = data.iloc[:, -1]
        class_names = ['Legitimate', 'Forgery']
    
    elif nome_dataset == 'wine_quality':
        data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", sep=';')
        X = data.drop(columns=['quality'])
        y = data['quality']
        class_names = sorted(y.unique().tolist())
    
    elif nome_dataset == 'heart_disease':
        data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data", header=None, na_values="?")
        data = data.dropna()  # Remove missing values
        X = data.iloc[:, :-1]
        y = data.iloc[:, -1]
        class_names = sorted(y.unique().tolist())
    
    elif nome_dataset == 'parkinsons':
        data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/parkinsons/parkinsons.data")
        X = data.drop(columns=['status', 'name'])
        y = data['status']
        class_names = ['Healthy', 'Parkinsons']
    
    elif nome_dataset == 'fashion_mnist':
        data = fetch_openml('Fashion-MNIST', version=1)
        X = pd.DataFrame(data.data)
        y = pd.Series(data.target).astype(int)
        class_names = [f"Class_{i}" for i in range(10)]
    
    elif nome_dataset == 'diabetes_binary':
        data = load_openml(name='diabetes')
        X = pd.DataFrame(data.data, columns=data.feature_names)
        y = pd.Series(data.target).apply(lambda x: 1 if x == 'tested_positive' else 0)
        class_names = ['Negative', 'Positive']
    
    else:
        raise ValueError("Nome do dataset não reconhecido. Escolha um dataset válido.")
    
    return X, y, class_names

# Menu de seleção de datasets
menu = '''
|  ************************* MENU ***************************  |
|  0 - iris                     |  1 - wine                     |
|  2 - breast_cancer            |  3 - digits                  |
|  4 - banknote_authentication  |  5 - wine_quality           |
|  6 - heart_disease            |  7 - parkinsons             |
|  8 - fashion_mnist            |  9 - diabetes_binary        |
|  Q - SAIR                                                |
|-------------------------------------------------------------|
'''

# Exibe o menu e solicita uma escolha
print(menu)
opcao = input("Digite o número do dataset ou 'Q' para sair: ").upper().strip()

# Processa a opção selecionada
while True:
    if opcao == 'Q':
        clear_output()
        print("Você escolheu sair.")
        break
    elif opcao.isdigit() and 0 <= int(opcao) <= 9:
        nomes_datasets = [
            'iris', 'wine', 'breast_cancer', 'digits', 'banknote_authentication',
            'wine_quality', 'heart_disease', 'parkinsons', 'fashion_mnist', 'diabetes_binary'
        ]
        nome_dataset = nomes_datasets[int(opcao)]
        clear_output()
        print(f"Dataset '{nome_dataset}' escolhido.")
        try:
            X, y, class_names = carregar_dataset(nome_dataset)
            print(f"Dataset {nome_dataset} carregado com sucesso.")
            print("Classes:", class_names)
            print("Amostras:", X.shape[0], "| Atributos:", X.shape[1])
            break
        except Exception as e:
            print(f"Erro ao processar o dataset: {e}")
    else:
        clear_output()
        print(menu)
        print("Opção inválida. Por favor, escolha um número do menu ou 'Q' para sair.")
        opcao = input("Digite o número do dataset ou 'Q' para sair: ").upper().strip()


Dataset 'wine' escolhido.
Dataset wine carregado com sucesso.
Classes: ['class_0' 'class_1' 'class_2']
Amostras: 178 | Atributos: 13


In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd

# Função para transformar o dataset em um problema binário (classe 0 contra as outras)
def transformar_problema_binario(y, classe_0):
    return [1 if label == classe_0 else 0 for label in y]

def analisar_instancias(X, y, class_names, classe_0=0, instancia_para_analisar=None):
    global TUDO
    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])

    # Transforma o problema em binário
    y_binario = transformar_problema_binario(y, classe_0)

    # Divide o dataset em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y_binario, test_size=0.2, random_state=42)
    
    # Treina o modelo
    modelo = LogisticRegression(max_iter=200)
    modelo.fit(X_train, y_train)

    # Obtém os nomes das features
    feature_names = X.columns.tolist()  
    
    # Seleciona as instâncias para análise
    num_instancias = len(X_test)
    instancias_para_analisar = range(num_instancias) if instancia_para_analisar is None else [instancia_para_analisar]
    
    TUDO = []
    # Loop para analisar cada instância selecionada
    for idx in instancias_para_analisar:
        Vs = X_test.iloc[idx].to_dict()
        instancia_test = X_test.iloc[[idx]]

        # Calcula `gamma_A` usando `decision_function`
        gamma_A = modelo.decision_function(instancia_test)[0]
        
        # Cálculo do valor delta para cada feature
        delta = []
        w = modelo.coef_[0]
        for i, feature in enumerate(feature_names):
            if w[i] < 0:
                delta.append((Vs[feature] - X[feature].max()) * w[i])
            else:
                delta.append((Vs[feature] - X[feature].min()) * w[i])

        # Calcula R
        R = sum(delta) - gamma_A
        
        # Computa a PI-explicação para a instância atual usando nomes das features
        Xpl = one_explanation(Vs, delta, R, feature_names, modelo, instancia_test, X)
        # Imprime os resultados
        classe_verdadeira = y_test[idx]
        print(f"\nInstância {idx}:")
        print(f"Classe verdadeira (binária): {classe_verdadeira}")
        print(f"PI-Explicação: ")
        
        TUDO.append(Xpl)

        for item in Xpl:
            print(f"- {item}")
            
            
        if not Xpl:
            print('_No-PI-explanation_'*3)
        

# Função para calcular a PI-explicação e incluir os intervalos de valores mínimos e máximos que garantem a classe
def one_explanation(Vs, delta, R, feature_names, modelo, instancia_test, X):
    Xpl = []
    delta_sorted = sorted(enumerate(delta), key=lambda x: abs(x[1]), reverse=True)
    R_atual = R
    Idx = 0
    
    while R_atual >= 0 and Idx < len(delta_sorted):
        sorted_idx, delta_value = delta_sorted[Idx]
        feature = feature_names[sorted_idx]
        feature_value = Vs[feature]

        # Encontra os valores mínimo e máximo para manter a classe usando perturbação
        #min_val, max_val = encontrar_intervalo_perturbacao(modelo, instancia_test, feature, feature_value, classe_desejada=1, X=X)

        # Adiciona a feature com o valor da instância e o intervalo mínimo/máximo que mantém a classe
        Xpl.append(f"{feature} - {feature_value} ")
        R_atual -= delta_value
        Idx += 1
    
    return Xpl

# Função para encontrar o intervalo de perturbação para manter a classe, considerando limites de X
def encontrar_intervalo_perturbacao(modelo, instancia, feature, valor_original, classe_desejada, X, passo=0.1, max_iter=50):
    # Define os valores mínimo e máximo baseados nos dados de entrada
    min_val_data = X[feature].min()
    max_val_data = X[feature].max()
    
    # Inicializa os valores mínimo e máximo com o valor da instância
    min_val, max_val = valor_original, valor_original
    
    # Perturba negativamente
    for _ in range(max_iter):
        min_val -= passo
        if min_val < min_val_data:
            min_val = min_val_data
            break
        instancia_perturbada = instancia.copy()
        instancia_perturbada[feature] = min_val
        predicao = modelo.predict(instancia_perturbada)
        if predicao[0] != classe_desejada:
            min_val += passo
            break

    # Perturba positivamente
    for _ in range(max_iter):
        max_val += passo
        if max_val > max_val_data:
            max_val = max_val_data
            break
        instancia_perturbada = instancia.copy()
        instancia_perturbada[feature] = max_val
        predicao = modelo.predict(instancia_perturbada)
        if predicao[0] != classe_desejada:
            max_val -= passo
            break

    return min_val, max_val

# Exemplo de uso (substitua X e y pelos dados adequados)
analisar_instancias(X, y, class_names, classe_0=0)
# Crie um dicionário para armazenar a contagem de cada feature
contagem_features = {}

# Itere sobre cada item da lista TUDO
for item in TUDO:
    # Verifique se o item é uma lista
    if isinstance(item, list):
        # Itere sobre cada item da lista
        for feature in item:
            # Extraia o nome da feature
            nome_feature = feature.split(" - ")[0]

            # Verifique se a feature já está no dicionário
            if nome_feature in contagem_features:
                # Incremente a contagem
                contagem_features[nome_feature] += 1
            else:
                # Adicione a feature ao dicionário com contagem 1
                contagem_features[nome_feature] = 1

# Imprima as features e suas contagens
for nome_feature, contagem in contagem_features.items():
    print(f"Feature: {nome_feature} Contagem: {contagem}")


Instância 0:
Classe verdadeira (binária): 1
PI-Explicação: 
- alcalinity_of_ash - 15.2 
- proline - 845.0 
- flavanoids - 3.03 
- ash - 2.56 
- magnesium - 116.0 

Instância 1:
Classe verdadeira (binária): 1
PI-Explicação: 
- proline - 1080.0 
- alcalinity_of_ash - 18.9 
- flavanoids - 2.65 
- malic_acid - 4.04 
- ash - 2.44 

Instância 2:
Classe verdadeira (binária): 0
PI-Explicação: 
- alcalinity_of_ash - 21.0 
- proline - 600.0 
- ash - 2.7 
- magnesium - 96.0 
- malic_acid - 2.81 
- color_intensity - 4.6 
- od280/od315_of_diluted_wines - 2.31 
- alcohol - 12.93 
- hue - 0.77 
- proanthocyanins - 0.75 
- flavanoids - 0.5 
- nonflavanoid_phenols - 0.53 
- total_phenols - 1.54 

Instância 3:
Classe verdadeira (binária): 1
PI-Explicação: 
- proline - 1285.0 
- alcalinity_of_ash - 22.5 
- flavanoids - 3.25 
- ash - 2.7 

Instância 4:
Classe verdadeira (binária): 0
PI-Explicação: 
- alcalinity_of_ash - 19.6 
- proline - 510.0 
- magnesium - 78.0 
- flavanoids - 2.0 
- od280/od315_of_dil

c:\Users\gleilsonpedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
